# FrameRate Independence and DeltaTime in Pygame 

<img src="img/having_different_framerates_is_a_problem.jpg" width="600px">

Players have different devices and that cause the thing that , maximum fps that a player might take changes depend on device to device. Considering this if we dont configure frame rate per second , all players that plays our  game wouldnt have the same experience because of the table that you can see above. 

> Greater fps rate causes player to move much more than the player that has lesser fps. 

Also Fps depends on which part of the game you are playing and this is a big problem even newest games like Lord Of The Rings. In LOTR , dungeons have 60 fps rate (which is bigger)  but in overworld part of the game players often get 30 fps. This makes players to complain about the experience that they gain in overworld.  

> Greater fps rates cause more often calculation of physical entities and more checkings for other kind of calculations. This makes the animations go smoothly and realisticly (which is of course a good thing). 

### Solution 1 : Determining a constant maximum Frame Rate for all players 

<img src="img/fr_problem_solution_1.jpg"  width = "600px">

### Solution 2 : Using Delta time for consistent speed at all frame rates

<img src="img/fr_problem_solution_2.jpg"  width ="600px">

#### How Delta Time Works ? 

<img src="img/delta_time_example_1.jpg" width = "600px">

<img src="img/delta_time_example_2.jpg" width = "600px">

#### What we should know before using Delta Time ? 

<img src="img/how2_use_delta_time.jpg" width="600px">

> Because of the multiplication with delta time (which is like 0.02 very small) you need to multiply this with larger amount of numbers to get faster effects 

#### How to get Delta Time in Python or in Pygame 

<img src="img/how2_get_dt_in_py.jpg" width="600px"> 

> Second approach is recommended because  it is more precise(more sensetive). 

#### First Method : Using `Dt = Clock.tick(framerate)` to get and set delta time

You can see the Dt value before configuration : 

In [1]:
import pygame  

pygame.init() 
font = pygame.font.Font(None , 30)

def debug(info , y = 10 , x = 10):  
    display_surface = pygame.display.get_surface() 
    debug_surf = font.render(str(info) , True , 'White') 
    debug_rect = debug_surf.get_rect(topleft = (x,y) ) 
    pygame.draw.rect(display_surface , 'Black' , debug_rect)  
    display_surface.blit(debug_surf , debug_rect) 


pygame 2.1.2 (SDL 2.0.18, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import pygame 

pygame.init() 
screen = pygame.display.set_mode((1000 , 600)) 
rect = pygame.Rect(50 , 300 , 50 , 50)   
test_speed = 5
clock = pygame.time.Clock()

running = True  
while running: 

    Dt = clock.tick(60)

    for e in pygame.event.get(): 
        if e.type == pygame.QUIT: 
            exit() 
            pygame.quit()  

    screen.fill((255,255,255)) 
    rect.centerx += test_speed 
    pygame.draw.rect(screen , (255,0,0) , rect) 
    debug(Dt) # clock.tick(x) method return 1000/x which is millisecond/frameRate 
    # But we want to use second/frame_Rate for delta_time (because of multiplication steps). So that we are gonna divide it with 1000 in the next
    # chapter
    pygame.display.update()

    

error: display Surface quit

: 

Dt with Configuration : 

In [2]:
import pygame 

pygame.init() 
screen = pygame.display.set_mode((1000 , 600)) 
rect = pygame.Rect(50 , 300 , 50 , 50)   
# test_speed = 5 Because of Delta Time is very low , moving object is very slow 
test_speed = 200
clock = pygame.time.Clock()

running = True  
while running: 

    # Dt = clock.tick(60)/1000 # (1000 millisecond/60) / 1000 = (second/60) moves smoothly with an normal speed 
    Dt = clock.tick(10)/1000 # moves with lag but with an normal speed 
    # So speed doesnt depend on Frame Rate anymore .

    for e in pygame.event.get(): 
        if e.type == pygame.QUIT: 
            exit() 
            pygame.quit()  

    screen.fill((255,255,255)) 
    rect.centerx += test_speed * Dt 
    pygame.draw.rect(screen , (255,0,0) , rect) 
    debug(Dt) 
    pygame.display.update()

error: display Surface quit

: 

#### Second Method : Using `Dt = time.time() - previous_time` to get and set Delta Time

<img src="img/delta_time_breaks_in_higher_frameRates.jpg" width = "600px">

> 1. Problem : While adding `rect.centerx += test_speed * Dt` the `test_speed * Dt` value becomes lesser than 1 more often and pygame cut decimals so it becomes `0` . That causes the object to not to move more often. But there is a slightly chance that this value nearly can be greater than 1 . On those times object moves .  

> 2. Problem : Cutting decimals makes the movement of rectangle unsystematic. That is what we want to avoid at all costs . 

##### Problematic Code in down bellow : 

In [1]:
import pygame  
import time

WIDTH , HEIGHT = 1000 , 600 
PURPLE = (217 , 0 , 104) 
PINK = (214 , 30 , 102) 
LIGHT_BLUE = (30, 102 , 204)

pygame.init()  

font = pygame.font.Font(None , 30)

def debug(info , y = 10 , x = 10):  
    display_surface = pygame.display.get_surface() 
    debug_surf = font.render(str(info) , True , 'White') 
    debug_rect = debug_surf.get_rect(topleft = (x,y) ) 
    pygame.draw.rect(display_surface , 'Black' , debug_rect)  
    display_surface.blit(debug_surf , debug_rect) 


screen = pygame.display.set_mode((WIDTH , HEIGHT))   

rect = pygame.Rect(50 , 300 , 100 , 100)  
uneffective_speed = 200 
uneffective_clock = pygame.time.Clock()


previous_time = time.time()  

running = True  
while running : 

    Dt = time.time() - previous_time  
    previous_time = time.time() 
    for e in pygame.event.get(): 
        if e.type == pygame.QUIT: 
            exit() 
            pygame.quit() 


    
    screen.fill(PURPLE) 
    rect.centerx += uneffective_speed * Dt 
    debug(uneffective_speed * Dt) # square moves unregularly 
    pygame.draw.rect(screen , LIGHT_BLUE , rect) 
    pygame.display.update() 
    

    

pygame 2.1.2 (SDL 2.0.18, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


error: display Surface quit

: 

### How to pretend these problems ? 

> 1. __Solution for Problem 1__: copy the `rect.centerx` value by assigning it to another variable called `test_rectx` and make the additition with this variable , e.g `test_rectx += (speed * delta_time)` . After that directly assign this consequance to the `rect.centerx` e.g 
`rect.centerx = test_rectx` 

>2. __Solution for Problem 2__ : instead of cutting the decimals off , we need to round the decimals to get better results. We do that with the `round(test_rectx)` method. 

In [1]:
import pygame  
import time

WIDTH , HEIGHT = 1000 , 600 
PURPLE = (217 , 0 , 104) 
PINK = (214 , 30 , 102) 
LIGHT_BLUE = (30, 102 , 204)

pygame.init()  

font = pygame.font.Font(None , 30)

def debug(info , y = 10 , x = 10):  
    display_surface = pygame.display.get_surface() 
    debug_surf = font.render(str(info) , True , 'White') 
    debug_rect = debug_surf.get_rect(topleft = (x,y) ) 
    pygame.draw.rect(display_surface , 'Black' , debug_rect)  
    display_surface.blit(debug_surf , debug_rect) 


screen = pygame.display.set_mode((WIDTH , HEIGHT))   

rect = pygame.Rect(50 , 300 , 100 , 100)  
uneffective_speed = 200 
uneffective_clock = pygame.time.Clock()

test_rect_x = rect.centerx

previous_time = time.time()  

running = True  
while running : 

    Dt = time.time() - previous_time  
    previous_time = time.time() 
    for e in pygame.event.get(): 
        if e.type == pygame.QUIT: 
            exit() 
            pygame.quit() 


    
    screen.fill(PURPLE) 
    test_rect_x += uneffective_speed * Dt  
    rect.centerx = round(test_rect_x)
    debug(round(test_rect_x)) # square moves regularly 
    pygame.draw.rect(screen , LIGHT_BLUE , rect) 
    pygame.display.update() 

pygame 2.1.2 (SDL 2.0.18, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


error: display Surface quit

: 

## Grand Project to Reinforce what we have learned : 

This will continue